In [1]:
from importlib import reload

In [2]:
import load_data

Loading up the load data and exploring the columns

In [3]:
loans = load_data.read_zipped_csv('./lending-club-loan-data.zip', 'loan.csv')
print(loans.columns)

C:\Users\Wball\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

Now, create a table model with all available columns.

In [4]:
import table_model
reload(table_model)

feature_columns = {
    'loan_amnt': table_model.NumericColumn(),
    'funded_amnt': table_model.NumericColumn(),
    'funded_amnt_inv': table_model.NumericColumn(),
    'term': table_model.CategoricalColumn(),
    'int_rate': table_model.PercentageColumn(),
    'installment': table_model.NumericColumn(),
    'grade': table_model.CategoricalColumn(),
    'sub_grade': table_model.CategoricalColumn(),
    'emp_length': table_model.CategoricalColumn(),
    'addr_state': table_model.CategoricalColumn(),
    'home_ownership': table_model.CategoricalColumn(),
    'annual_inc': table_model.NumericColumn(),
    'annual_inc_joint': table_model.NumericColumn(),
    'dti': table_model.NumericColumn(),
    'dti_joint': table_model.NumericColumn(),
    'open_acc': table_model.NumericColumn(),
    'total_acc': table_model.NumericColumn(),
    'pub_rec': table_model.NumericColumn(),
    'tot_coll_amt': table_model.NumericColumn(),
    'total_bal_il': table_model.NumericColumn(),
    'tot_cur_bal': table_model.NumericColumn(),
    'revol_bal': table_model.NumericColumn(),
    'revol_util': table_model.PercentageColumn(),
    'mths_since_last_delinq': table_model.NumericColumn(),
    'mths_since_last_record': table_model.NumericColumn(),
    'mths_since_last_major_derog': table_model.NumericColumn(),
    'inq_fi': table_model.NumericColumn(),
    'inq_last_12m': table_model.NumericColumn(),
    'all_util': table_model.NumericColumn(),
    'acc_now_delinq': table_model.NumericColumn(),
    # and the output
    'loan_status': table_model.OutputLabelColumn(),
}

loan_model = table_model.TableModel(feature_columns, 'loan_status')
filtered = loans.loc[loans.loan_status.isin(['Current', 'Default'])]
loan_model.fit(filtered)
x, y = loan_model.transform(filtered)

Using TensorFlow backend.


In [5]:
print(x, x.dtype)

[[ 0.05882353  0.05882353  0.06158358 ...,  0.11111111  0.          0.        ]
 [ 0.17647059  0.17647059  0.17888562 ...,  0.11111111  0.          0.        ]
 [ 0.2647059   0.2647059   0.26612905 ...,  0.11111111  0.          0.        ]
 ..., 
 [ 0.35294119  0.35294119  0.3548387  ...,  0.11111111  0.          0.        ]
 [ 0.32352942  0.32352942  0.32551318 ...,  0.11111111  0.          0.        ]
 [ 0.55882353  0.55882353  0.5601173  ...,  0.11111111  0.          0.        ]] float32


In [6]:
print(y, y.dtype)

[0 0 0 ..., 0 0 0] int32


In [11]:
import keras
import numpy as np

keras.backend.clear_session()

model = keras.models.Sequential()
y_one_hot = keras.utils.to_categorical(y).astype(np.float32)
# logistic regression is a one layer model
model.add(keras.layers.Dense(y_one_hot.shape[1], activation='sigmoid', input_dim=x.shape[1]))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(x, y_one_hot, epochs=4, batch_size=256)

Epoch 1/4
602998/602998 [==============================] - 15s 24us/step - loss: 0.0605
Epoch 2/4
602998/602998 [==============================] - 15s 24us/step - loss: 0.0148
Epoch 3/4
602998/602998 [==============================] - 17s 28us/step - loss: 0.0144
Epoch 4/4
602998/602998 [==============================] - 15s 25us/step - loss: 0.0142


In [12]:
predictions = model.predict_classes(x)

In [13]:
import sklearn.metrics
print(sklearn.metrics.accuracy_score(y, predictions))

0.997978434423


Looks good -- but there is probably a catch here, looking at the balance of classes, far more loans are 'Current' than 'Default', so our model probaly just learned to guess 'Current'.

In [14]:
import numpy as np
labels, counts = np.unique(y, return_counts=True)
print(labels, counts)

[0 1] [601779   1219]


In [15]:
print(1 - (counts[1] / counts[0]))

0.997974339417


So -- these highly imbalanced classes will make it hard to predict 'Default', let's see it that was the case.

In [16]:
print(np.unique(predictions, return_counts=True))

(array([0], dtype=int64), array([602998], dtype=int64))


In [17]:
print(sklearn.metrics.classification_report(y, predictions, target_names=loan_model.classes))

             precision    recall  f1-score   support

    Current       1.00      1.00      1.00    601779
    Default       0.00      0.00      0.00      1219

avg / total       1.00      1.00      1.00    602998



C:\Users\Wball\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


So -- clearly the model isn't actually very useful, even though it is accurate! 
Real data often needs to predict relatively rare events.

In [19]:
reload(table_model)

logistic_regression_model = table_model.KerasLogisticRegressionModel()
logistic_regression_model.fit(x, y)

Epoch 1/16
602998/602998 [==============================] - 15s 25us/step - loss: 1.3299
Epoch 2/16
602998/602998 [==============================] - 16s 26us/step - loss: 1.2581
Epoch 3/16
602998/602998 [==============================] - 16s 27us/step - loss: 1.2419
Epoch 4/16
602998/602998 [==============================] - 15s 25us/step - loss: 1.2348
Epoch 5/16
602998/602998 [==============================] - 12s 20us/step - loss: 1.2311
Epoch 6/16
602998/602998 [==============================] - 14s 24us/step - loss: 1.2288
Epoch 7/16
602998/602998 [==============================] - 16s 27us/step - loss: 1.2278
Epoch 8/16
602998/602998 [==============================] - 15s 25us/step - loss: 1.2258
Epoch 9/16
602998/602998 [==============================] - 16s 26us/step - loss: 1.2253
Epoch 10/16
602998/602998 [==============================] - 15s 25us/step - loss: 1.2247
Epoch 11/16
602998/602998 [==============================] - 16s 26us/step - loss: 1.2238
Epoch 12/16
602998/

KerasLogisticRegressionModel(epochs=16, verbose=1)

In [20]:
balanced_predictions = logistic_regression_model.predict(x)

In [21]:
print(sklearn.metrics.accuracy_score(y, balanced_predictions))

0.630786503438


In [22]:
print(np.unique(balanced_predictions, return_counts=True))

(array([0, 1], dtype=int64), array([379906, 223092], dtype=int64))


So - the accuracy went down, but the model is at leasty trying to predict 'Default'!

In [23]:
print(sklearn.metrics.classification_report(y, balanced_predictions, target_names=loan_model.classes))

             precision    recall  f1-score   support

    Current       1.00      0.63      0.77    601779
    Default       0.00      0.69      0.01      1219

avg / total       1.00      0.63      0.77    602998



Now - to try a deep learning model.

In [24]:
reload(table_model)
deep_model = table_model.KerasDeepClassifierModel()
deep_model.fit(x, y)

Epoch 1/16
602998/602998 [==============================] - 30s 50us/step - loss: 1.4323
Epoch 2/16
602998/602998 [==============================] - 30s 50us/step - loss: 1.2423
Epoch 3/16
602998/602998 [==============================] - 30s 50us/step - loss: 1.1991
Epoch 4/16
602998/602998 [==============================] - 29s 48us/step - loss: 1.1502
Epoch 5/16
602998/602998 [==============================] - 31s 52us/step - loss: 1.1038
Epoch 6/16
602998/602998 [==============================] - 29s 49us/step - loss: 1.0417
Epoch 7/16
602998/602998 [==============================] - 29s 49us/step - loss: 0.9738
Epoch 8/16
602998/602998 [==============================] - 29s 49us/step - loss: 0.9265
Epoch 9/16
602998/602998 [==============================] - 29s 48us/step - loss: 0.8623
Epoch 10/16
602998/602998 [==============================] - 27s 44us/step - loss: 0.8094
Epoch 11/16
602998/602998 [==============================] - 28s 46us/step - loss: 0.7589
Epoch 12/16
602998/

KerasDeepClassifierModel(batch_size=256, epochs=16, verbose=1)

In [25]:
deep_predictions = deep_model.predict(x)

In [26]:
print(sklearn.metrics.accuracy_score(y, deep_predictions))

0.833684357162


In [27]:
print(np.unique(deep_predictions, return_counts=True))

(array([0, 1], dtype=int64), array([501567, 101431], dtype=int64))


In [28]:
print(sklearn.metrics.classification_report(y, deep_predictions, target_names=loan_model.classes))

             precision    recall  f1-score   support

    Current       1.00      0.83      0.91    601779
    Default       0.01      0.97      0.02      1219

avg / total       1.00      0.83      0.91    602998



And now -- combining both of these techniques in Wide + Deep models.

In [44]:
reload(table_model)
deep_model = table_model.KerasWideAndDeepClassifierModel(epochs=64)
deep_model.fit(x, y)

Epoch 1/64
602998/602998 [==============================] - 19s 32us/step - loss: 1.3389
Epoch 2/64
602998/602998 [==============================] - 30s 50us/step - loss: 1.2860
Epoch 3/64
602998/602998 [==============================] - 32s 53us/step - loss: 1.2751
Epoch 4/64
602998/602998 [==============================] - 25s 41us/step - loss: 1.2565
Epoch 5/64
602998/602998 [==============================] - 33s 54us/step - loss: 1.2535 0s - loss: 1.255 - ETA: 0s - loss: 1
Epoch 6/64
602998/602998 [==============================] - 34s 57us/step - loss: 1.2382
Epoch 7/64
602998/602998 [==============================] - 32s 54us/step - loss: 1.2332
Epoch 8/64
602998/602998 [==============================] - 31s 51us/step - loss: 1.2337
Epoch 9/64
602998/602998 [==============================] - 33s 54us/step - loss: 1.2152
Epoch 10/64
602998/602998 [==============================] - 27s 44us/step - loss: 1.2004
Epoch 11/64
602998/602998 [==============================] - 18s 30us/st

KerasWideAndDeepClassifierModel(batch_size=512, epochs=64, verbose=1)

In [45]:
wd_predictions = deep_model.predict(x)

In [46]:
print(sklearn.metrics.accuracy_score(y, wd_predictions))

0.877037734785


In [47]:
print(np.unique(wd_predictions, return_counts=True))

(array([0, 1], dtype=int64), array([527975,  75023], dtype=int64))


In [48]:
print(sklearn.metrics.classification_report(y, wd_predictions, target_names=loan_model.classes))

             precision    recall  f1-score   support

    Current       1.00      0.88      0.93    601779
    Default       0.01      0.86      0.03      1219

avg / total       1.00      0.88      0.93    602998

